In [1]:
!pip install avalanche-lib==0.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 811 kB 30.0 MB/s 
     |████████████████████████████████| 512 kB 71.6 MB/s 
     |████████████████████████████████| 1.9 MB 74.3 MB/s 
     |████████████████████████████████| 532 kB 76.2 MB/s 
     |████████████████████████████████| 455 kB 75.4 MB/s 
     |████████████████████████████████| 174 kB 72.0 MB/s 
     |████████████████████████████████| 184 kB 63.3 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 173 kB 75.4 MB/s 
     |████████████████████████████████| 168 kB 74.3 MB/s 
     |████████████████████████████████| 168 kB 78.9 MB/s 
     |████████████████████████████████| 166 kB 86.3 MB/s 
     |████████████████████████████████| 166 kB 63.2 MB/s 
     |████████████████████████████████| 162 kB 83.3 MB/s 
     |████████████████████████████████| 162 kB 84.3 MB/s 
     |███████████████████████████

In [18]:
import torch
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST, PermutedMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.supervised import Naive, LwF, EWC, GenerativeReplay

device = torch.device(
        f"cuda:0"
        if torch.cuda.is_available()
        else "cpu"
    )

In [14]:
scenario = SplitMNIST(n_experiences=5)

In [12]:
scenario = PermutedMNIST(n_experiences=5)

Naive:

In [15]:
# MODEL CREATION
model = SimpleMLP(num_classes=scenario.n_classes)

# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    #timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    #cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario.n_classes, save_image=False,
                             stream=True),
    #disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)

# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=500, train_epochs=5, eval_mb_size=100,
    evaluator=eval_plugin)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 3]
-- >> Start of training phase << --
100%|██████████| 25/25 [00:01<00:00, 13.34it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.1007
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6984
100%|██████████| 25/25 [00:01<00:00, 13.80it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1583
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9687
100%|██████████| 25/25 [00:01<00:00, 13.29it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0966
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9754
100%|██████████| 25/25 [00:02<00:00, 11.14it/s]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0806
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9783
100%|██████████| 25/25 [00:01<00:00, 13.19it/s]
Epoch 4 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0689
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9813
-- >> End of trai

LwF:

In [21]:
# MODEL CREATION
model = SimpleMLP(num_classes=scenario.n_classes, hidden_size=256)

# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    #timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    #cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario.n_classes, save_image=False,
                             stream=True),
    #disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)

# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = LwF(
        model,
        SGD(model.parameters(), lr=0.001),
        CrossEntropyLoss(),
        alpha=[0, 0.5, 1.333, 2.25, 3.2],
        temperature=1,
        train_epochs=10,
        train_mb_size=128,
        evaluator=eval_plugin,
    )

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 3]
-- >> Start of training phase << --
100%|██████████| 95/95 [00:02<00:00, 42.37it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.3884
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7013
100%|██████████| 95/95 [00:02<00:00, 46.08it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4373
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9555
100%|██████████| 95/95 [00:02<00:00, 46.04it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2537
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9660
100%|██████████| 95/95 [00:02<00:00, 45.35it/s]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1878
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9701
100%|██████████| 95/95 [00:02<00:00, 46.52it/s]
Epoch 4 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1533
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9709
100%|██████████| 

EwC:

In [17]:
# MODEL CREATION
model = SimpleMLP(num_classes=scenario.n_classes, hidden_size=256)

# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    #timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    #cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario.n_classes, save_image=False,
                             stream=True),
    #disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)

# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = EWC(
        model,
        SGD(model.parameters(), lr=0.001),
        CrossEntropyLoss(),
        0.4,
        "separate",
        #decay_factor=0.1,
        train_epochs=5,
        device=device,
        train_mb_size=128,
        evaluator=eval_plugin,
    )

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 3]
-- >> Start of training phase << --
100%|██████████| 95/95 [00:02<00:00, 46.30it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.3615
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6789
100%|██████████| 95/95 [00:02<00:00, 43.59it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4941
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9362
100%|██████████| 95/95 [00:02<00:00, 44.72it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2940
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9536
100%|██████████| 95/95 [00:02<00:00, 33.12it/s]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2123
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9646
100%|██████████| 95/95 [00:03<00:00, 31.23it/s]
Epoch 4 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1709
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9691
-- >> End of trai

Generative Replay:

In [19]:
# MODEL CREATION
model = SimpleMLP(num_classes=scenario.n_classes, hidden_size=256)

# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    #timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    #cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario.n_classes, save_image=False,
                             stream=True),
    #disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)

# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = GenerativeReplay(
      model,
      torch.optim.Adam(model.parameters(), lr=0.001),
      CrossEntropyLoss(),
      train_mb_size=128,
      train_epochs=5,
      eval_mb_size=128,
      replay_size=100,
      device=device,
      evaluator=eval_plugin,
  )

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 3]
-- >> Start of training phase << --
100%|██████████| 95/95 [00:02<00:00, 38.67it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0870
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9746
100%|██████████| 95/95 [00:02<00:00, 45.87it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0190
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9929
100%|██████████| 95/95 [00:02<00:00, 44.86it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0149
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9950
100%|██████████| 95/95 [00:02<00:00, 43.82it/s]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0102
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9971
100%|██████████| 95/95 [00:02<00:00, 46.65it/s]
Epoch 4 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0081
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9973
-- >> Start of tr